In [2]:
from MQGO import data, main, ML_engine, optimizer, QC_engine 

In [3]:
from deepmd.calculator import DP

2022-03-24 15:24:46.637384: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 15:24:46.637428: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
setting = {'basis':'6-31g', 'spin':0, 'xc':None, 'restricted':True, 'verbose':4}
geo = main.MLgeomopt(qc_engine='pyscf', ml_engine='deepmd', work_path='./', xyz_path='./water.xyz',**setting)
geo.opt_algorithm = 'pyberny'
geo.kernel()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-12
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/jingheng/ML_QC_GEOM_OPT/test/water/dimer/tmpsio3gvlo
max_memory 4000 MB (current use 356 MB)
Set gradient conv threshold to 1e-06
init E= -151.774783541679
  HOMO = -0.484751289552413  LUMO = 0.0941205951443397
cycle= 1 E= -151.825966846584  delta_E= -0.0512  |g|= 0.605  |ddm|= 1.71
  HOMO = -0.39246942555559  LUMO = 0.21599425419948
cycle= 2 E= -151.871882391472  delta_E= -0.0459  |g|= 0.312  |ddm|= 0.494
  HOMO = -0.477213847546512  LUMO = 0.20192340693829
cycle= 3 E= -151.889170937807  delta_E= -0.0173  |g|= 0.0216  |ddm|= 0.182
  HOMO = -0.472288384744339  LUMO = 0.205512303603992
cycle= 4 E= -151.889268185819  delta_E= -9.72e-05  |g|= 0.00105  |d

OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-15
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #216: KMP_AFFINITY: x2APIC ids not unique.
OMP: Info #216: KMP_AFFINITY: decoding legacy APIC ids.
OMP: Info #216: KMP_AFFINITY: legacy APIC ids not unique.
OMP: Info #220: KMP_AFFINITY: parsing /proc/cpuinfo.
OMP: Info #157: KMP_AFFINITY: 16 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #192: KMP_AFFINITY: 1 socket x 8 cores/socket x 2 threads/core (8 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 2 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 3 maps to socket 0 core 1 thread 1 
OMP: Info #172: KMP_AFFINITY: OS p

******** <class 'pyscf.grad.rhf.Gradients'> for <class 'pyscf.scf.hf.RHF'> ********
unit = Eh/Bohr
max_memory 4000 MB (current use 362 MB)
--------------- RHF gradients ---------------
         x                y                z
0 O     0.0000032134     0.0000026407     0.0000001046
1 O     0.0000026887    -0.0000005090     0.0000002209
2 H    -0.0019738981     0.0009284966    -0.0000009582
3 H     0.0019706551    -0.0009311433     0.0000008505
4 H    -0.0000773182    -0.0001792205    -0.0021718480
5 H     0.0000746592     0.0001797355     0.0021716301
----------------------------------------------
main.py 81 QC_coords0: [[ 3.4108076   1.93082821  1.75628343]
 [26.18307531  1.18536426  1.76058627]
 [ 4.2458177   1.53713687  1.75666035]
 [ 2.57580473  2.32453493  1.75590623]
 [26.2152549   1.26134719  2.68005673]
 [26.15088978  1.10936856  0.84111703]]
main.py 82 E_QC0: -4133.117517090819
main.py 83 G_QC0: [[ 1.65238089e-04  1.35788408e-04  5.38051564e-06]
 [ 1.38256587e-04 -2.61738263

2022-03-24 15:24:49.832314: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-24 15:24:49.832354: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
/home/jingheng/venv/lib/python3.9/site-packages/deepmd/utils/data_system.py:158: UserWarning: system ./raw/ required batch size is larger than the size of the dataset raw/set.000 (6 > 1)
  warnings.warn("system %s required batch size is larger than the size of the dataset %s (%d > %d)" % \
2022-03-24 15:24:53.438130: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Ne

optimizer.py 77: [[ 3.63802  1.5711   1.76   ]
 [26.36198  1.5711   1.76   ]
 [ 3.95828  2.55412  1.74392]
 [ 2.63613  1.66728  1.76493]
 [26.09174  0.99418  2.56104]
 [26.0955   0.9908   0.96072]]


...

       Step     Time          Energy          fmax
Berny:    0 15:27:16    -4133.135702         0.112157
Berny:    1 15:27:16    -4133.136947         0.112908
Berny:    2 15:27:16    -4133.170915         0.140651
Berny:    3 15:27:16    -4133.247230         0.303205
Berny:    4 15:27:16    -4133.406822         1.115714
Berny:    5 15:27:16    -4133.957938         1.179881
Berny:    6 15:27:16    -4134.296278         2.028092
Berny:    7 15:27:16    -4137.131886        11.519892
Berny:    8 15:27:16    -4141.291528        18.225557
Berny:    9 15:27:16    -4145.007256       337.476458
Berny:   10 15:27:16    -4144.814663       236.135420
Berny:   11 15:27:16    -4143.338576        20.395648
Berny:   12 15:27:16    -4143.988000        49.119533
Berny:   13 15:27:16    -4144.499947        44.257229
Berny:   14 15:27:16    -4144.824750        74.714735
Berny:   15 15:27:16    -4144.858300       125.138835
Berny:   16 15:27:16    -4144.941749       277.793116
Berny:   17 15:27:16    -4144.9

OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10411 thread 16 bound to OS proc set 0
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10414 thread 18 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10413 thread 17 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10415 thread 19 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10416 thread 20 bound to OS proc set 8
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10417 thread 21 bound to OS proc set 10
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10419 thread 23 bound to OS proc set 14
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10418 thread 22 bound to OS proc set 12
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10420 thread 24 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10421 thread 25 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10422 thread 26 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 9660 tid 10423 thread 27 bound to OS proc set 7
OMP: Info #25

Berny:   18 15:27:16    -4144.997699       342.271832
Berny:   19 15:27:16    -4145.003151       341.149836
Berny:   20 15:27:16    -4145.006283       338.630849


/home/jingheng/anaconda3/lib/python3.9/site-packages/berny/Math.py:135: RuntimeWarning: divide by zero encountered in double_scalars
  x = x - fx / dxf
/home/jingheng/anaconda3/lib/python3.9/site-packages/berny/berny.py:243: RuntimeWarning: invalid value encountered in multiply
  return norm(np.linalg.solve(l * eye(H.shape[0]) - H, g)) - trust


FindrootException: 